In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd
import re

In [2]:
def add_multicolumn(df, df2, new_col_name):
    tmp = df2.copy()    # make copy, otherwise df2 will be changed !!!
    tmp.columns = pd.MultiIndex.from_product([[new_col_name], df2.columns.tolist()])
    return pd.concat([df, tmp], axis=1)

def get_valid_filename(s):
    s = str(s).strip().replace(' ', '_')
    return re.sub(r'(?u)[^-\w.]', '', s)

In [3]:
model_path = '/users/Stathis/GoogleDrive/MLP/transfer_all_layers/'
nets =  list(filter(lambda x: os.path.isdir(os.path.join(model_path, x)), os.listdir(model_path)))
nets = [a for a in nets if 'resnet56' in a  or 'densenet121' in a]
nets.sort()

li = []
for net in nets:
    filename =  os.path.join(model_path, net) +'/result_outputs/test_summary.csv'
    df = pd.read_csv(filename, delimiter=",", index_col=None, header=0)
    nn = net.split('_')
    df['net'] = nn[3]
    df['source'] = nn[4]
    df['target'] = nn[5]
    df['step'] = nn[7]
    df['gamma'] = nn[9]
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [30]:
frame.set_index(['net', 'source', 'target'])

test_acc  test_loss step gamma
net         source target                                
densenet121 fgsm   fgsm      0.9145   0.358316   25   0.1
                   nat       0.9401   0.256466   25   0.1
                   nat       0.9388   0.254179   25   0.1
            nat    fgsm      0.9035   0.338362   25   0.1
                   nat       0.9340   0.271384   25   0.1
                   nat       0.9351   0.260235   25   0.1
                   pgd       0.8945   0.336705   25   0.1
            pgd    nat       0.9277   0.268576   25   0.1
                   nat       0.9261   0.296056   25   0.1
                   pgd       0.8938   0.357319   25   0.1
resnet56    fgsm   fgsm      0.8929   0.356749   25   0.1
                   fgsm      0.8906   0.370036   25   0.1
                   nat       0.9178   0.315059   25   0.1
                   nat       0.9189   0.355595   25   0.1
            nat    fgsm      0.9052   0.308398   25   0.1
                   fgsm      0.8925   0.355220   25   0.1
                   nat       0.9303   0.307776   20   0.4
                   nat       0.9250   0.324039   25   0.1
                   pgd       0.8810   0.439584   25   0.1
            pgd    nat       0.9184   0.314190   25   0.1
                   nat       0.9128   0.355118   25   0.1
                   pgd       0.8868   0.372155   25   0.1

In [53]:
import os
import json

# Label Leaking

In [80]:
model_path = '/users/Stathis/GoogleDrive/MLP/attack_results_pgd_7/'
nets =  list(os.listdir(model_path))
nets = [a for a in nets if 'all_layers' in a and 'no_LL' not in a]
nets.sort()

attack_results = {}

for net in nets:
    fname = model_path+net
    with open(fname,'r+') as json_data:
        attack_results[net[24:-5]] = list(json.load(json_data).values())
        
df = pd.DataFrame.from_dict(attack_results, columns=['Clean', 
                                                     'FGSM_0.03125', 'PGD_0.03125',
                                                     'FGSM_0.0625', 'PGD_0.0625',
                                                     'FGSM_0.125', 'PGD_0.125',                                                    
                                                    ], orient='index')
df=np.round(df*100,1)

for net in df.iterrows():
#     print(net[0])
    nn = net[0].split('_')
#     print(nn)
    df.loc[net[0],'net'] = nn[3]
    df.loc[net[0],'source'] = nn[4]
    df.loc[net[0],'target'] = nn[5]
    df.loc[net[0],'step'] = nn[7]
    df.loc[net[0],'gamma'] = nn[9]
    
df.groupby([ 'net','source', 'target', ]).mean()

Clean  FGSM_0.03125  PGD_0.03125  FGSM_0.0625  \
net         source target                                                  
densenet121 fgsm   fgsm    91.50         77.20        60.30        68.30   
                   nat     93.95         60.50        19.80        48.85   
            nat    fgsm    90.40         78.80        65.00        69.80   
                   nat     93.45         56.20        14.75        45.05   
                   pgd     89.40         75.40        60.50        66.40   
            pgd    nat     92.70         60.05        21.35        48.80   
                   pgd     89.40         77.60        60.30        67.20   
resnet56    fgsm   fgsm    89.20         72.65        54.95        59.80   
                   nat     91.85         60.05        22.30        48.75   
            nat    fgsm    89.90         76.75        66.10        66.30   
                   nat     92.75         58.45        16.80        46.65   
                   pgd     88.10         76.50        65.10        68.10   
            pgd    nat     91.55         61.65        21.05        51.40   
                   pgd     88.70         75.70        60.00        66.70   

                           PGD_0.0625  FGSM_0.125  PGD_0.125  
net         source target                                     
densenet121 fgsm   fgsm         36.00       54.20      13.80  
                   nat           6.85       37.90       4.00  
            nat    fgsm         41.80       53.40      17.20  
                   nat           5.70       34.95       3.80  
                   pgd          35.80       52.80      14.20  
            pgd    nat           7.90       37.40       4.75  
                   pgd          35.70       52.10      13.90  
resnet56    fgsm   fgsm         29.55       44.55      12.40  
                   nat           8.80       38.15       5.35  
            nat    fgsm         44.10       49.40      19.30  
                   nat           6.65       36.20       4.55  
                   pgd          42.90       52.90      19.30  
            pgd    nat           9.20       40.60       5.70  
                   pgd          35.80       52.30      15.30

# No Label Leaking

In [76]:
model_path = '/users/Stathis/GoogleDrive/MLP/attack_results_pgd_7/'
nets =  list(os.listdir(model_path))
nets = [a for a in nets if 'all_layers' in a and 'no_LL' in a]
nets.sort()

attack_results = {}

for net in nets:
    fname = model_path+net
    with open(fname,'r+') as json_data:
        attack_results[net[30:-5]] = list(json.load(json_data).values())
        
df = pd.DataFrame.from_dict(attack_results, columns=['Clean', 
                                                     'FGSM_0.03125', 'PGD_0.03125',
                                                     'FGSM_0.0625', 'PGD_0.0625',
                                                     'FGSM_0.125', 'PGD_0.125',                                                    
                                                    ], orient='index')
df=np.round(df*100,1)

for net in df.iterrows():
#     print(net[0])
    nn = net[0].split('_')
#     print(nn)
    df.loc[net[0],'net'] = nn[3]
    df.loc[net[0],'source'] = nn[4]
    df.loc[net[0],'target'] = nn[5]
    df.loc[net[0],'step'] = nn[7]
    df.loc[net[0],'gamma'] = nn[9]
    
# df.set_index([ 'net','source', 'target', ])

In [82]:
df.groupby([ 'net','source', 'target', ]).mean()

Clean  FGSM_0.03125  PGD_0.03125  FGSM_0.0625  \
net         source target                                                  
densenet121 fgsm   fgsm    91.50         77.20        60.30        68.30   
                   nat     93.95         60.50        19.80        48.85   
            nat    fgsm    90.40         78.80        65.00        69.80   
                   nat     93.45         56.20        14.75        45.05   
                   pgd     89.40         75.40        60.50        66.40   
            pgd    nat     92.70         60.05        21.35        48.80   
                   pgd     89.40         77.60        60.30        67.20   
resnet56    fgsm   fgsm    89.20         72.65        54.95        59.80   
                   nat     91.85         60.05        22.30        48.75   
            nat    fgsm    89.90         76.75        66.10        66.30   
                   nat     92.75         58.45        16.80        46.65   
                   pgd     88.10         76.50        65.10        68.10   
            pgd    nat     91.55         61.65        21.05        51.40   
                   pgd     88.70         75.70        60.00        66.70   

                           PGD_0.0625  FGSM_0.125  PGD_0.125  
net         source target                                     
densenet121 fgsm   fgsm         36.00       54.20      13.80  
                   nat           6.85       37.90       4.00  
            nat    fgsm         41.80       53.40      17.20  
                   nat           5.70       34.95       3.80  
                   pgd          35.80       52.80      14.20  
            pgd    nat           7.90       37.40       4.75  
                   pgd          35.70       52.10      13.90  
resnet56    fgsm   fgsm         29.55       44.55      12.40  
                   nat           8.80       38.15       5.35  
            nat    fgsm         44.10       49.40      19.30  
                   nat           6.65       36.20       4.55  
                   pgd          42.90       52.90      19.30  
            pgd    nat           9.20       40.60       5.70  
                   pgd          35.80       52.30      15.30

# Black box attacks

In [74]:
js = json.load(open("../experiments_results/attack_results/black_box_results_all_layers.json"))

In [75]:
js

{'cifar10_e_0.03125': [{'transfer_all_layers_densenet121_fgsm_fgsm_step_25_gamma_0.1_attacked_by_fgsm_acc': 0.8423999999999999,
   'transfer_all_layers_densenet121_fgsm_nat_step_25_gamma_0.1_attacked_by_fgsm_acc': 0.7350000000000001,
   'transfer_all_layers_densenet121_fgsm_nat_step_25_gamma_0.1_2_attacked_by_fgsm_acc': 0.7302999999999998,
   'transfer_all_layers_densenet121_nat_fgsm_step_25_gamma_0.1_attacked_by_fgsm_acc': 0.8525999999999998,
   'transfer_all_layers_densenet121_nat_nat_step_25_gamma_0.1_attacked_by_fgsm_acc': 0.6912999999999999,
   'transfer_all_layers_densenet121_nat_nat_step_25_gamma_0.1_2_attacked_by_fgsm_acc': 0.6949000000000001,
   'transfer_all_layers_densenet121_nat_pgd_step_25_gamma_0.1_attacked_by_fgsm_acc': 0.8153,
   'transfer_all_layers_densenet121_pgd_nat_step_25_gamma_0.1_attacked_by_fgsm_acc': 0.7225,
   'transfer_all_layers_densenet121_pgd_nat_step_25_gamma_0.1_2_attacked_by_fgsm_acc': 0.7249999999999999,
   'transfer_all_layers_densenet121_pgd_pgd_ste